<a href="https://colab.research.google.com/github/sobit-nep/Transformer-Neural-Network-from-scratch/blob/main/positional_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn

max_sequence_length = 10 #for my case
d_model = 6 #typically 512

# Positional Embedding

PE (pos, 2i) = sin(pos/10000^(2i/d_model))

PE (pos,2i+1) = cos(pos/10000^(2i/d_model))

i= dimension index

d_model = Embedding Length

pos = Position of word in sequence

WHY?
1. Periodicity
2. Constrained Values
3. Easy to extrapolate for long sequences

PE (pos, i) = sin(pos/10000^(i/d_model)) when i is even

PE (pos,i+1) = cos(pos/10000^(i-1/d_model)) when i is odd

In [4]:
even_i = torch.arange(0,d_model, 2).float()
even_i

tensor([0., 2., 4.])

In [5]:
even_denominator = torch.pow(1000,even_i/d_model)
even_denominator

tensor([  1.0000,  10.0000, 100.0000])

In [6]:
odd_i = torch.arange(1, d_model, 2).float()
odd_i

tensor([1., 3., 5.])

In [7]:
odd_denominator = torch.pow(1000,(odd_i-1)/d_model)
odd_denominator

tensor([  1.0000,  10.0000, 100.0000])

Here we can see that odd_denominator=even_denominator.

In [8]:
denominator = even_denominator

In [9]:
position = torch.arange(max_sequence_length,dtype=torch.float).reshape(max_sequence_length,1)

In [10]:
position

tensor([[0.],
        [1.],
        [2.],
        [3.],
        [4.],
        [5.],
        [6.],
        [7.],
        [8.],
        [9.]])

In [11]:
even_PE = torch.sin(position/denominator)
odd_PE = torch.cos(position/denominator)

In [12]:
even_PE

tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.8415,  0.0998,  0.0100],
        [ 0.9093,  0.1987,  0.0200],
        [ 0.1411,  0.2955,  0.0300],
        [-0.7568,  0.3894,  0.0400],
        [-0.9589,  0.4794,  0.0500],
        [-0.2794,  0.5646,  0.0600],
        [ 0.6570,  0.6442,  0.0699],
        [ 0.9894,  0.7174,  0.0799],
        [ 0.4121,  0.7833,  0.0899]])

In [13]:
odd_PE

tensor([[ 1.0000,  1.0000,  1.0000],
        [ 0.5403,  0.9950,  0.9999],
        [-0.4161,  0.9801,  0.9998],
        [-0.9900,  0.9553,  0.9996],
        [-0.6536,  0.9211,  0.9992],
        [ 0.2837,  0.8776,  0.9988],
        [ 0.9602,  0.8253,  0.9982],
        [ 0.7539,  0.7648,  0.9976],
        [-0.1455,  0.6967,  0.9968],
        [-0.9111,  0.6216,  0.9960]])

In [14]:
odd_PE.shape

torch.Size([10, 3])

In [15]:
stacked = torch.stack([even_PE,odd_PE], dim=2)
stacked.shape

torch.Size([10, 3, 2])

In [16]:
PE = torch.flatten(stacked, start_dim=1, end_dim=2)
PE

tensor([[ 0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.8415,  0.5403,  0.0998,  0.9950,  0.0100,  0.9999],
        [ 0.9093, -0.4161,  0.1987,  0.9801,  0.0200,  0.9998],
        [ 0.1411, -0.9900,  0.2955,  0.9553,  0.0300,  0.9996],
        [-0.7568, -0.6536,  0.3894,  0.9211,  0.0400,  0.9992],
        [-0.9589,  0.2837,  0.4794,  0.8776,  0.0500,  0.9988],
        [-0.2794,  0.9602,  0.5646,  0.8253,  0.0600,  0.9982],
        [ 0.6570,  0.7539,  0.6442,  0.7648,  0.0699,  0.9976],
        [ 0.9894, -0.1455,  0.7174,  0.6967,  0.0799,  0.9968],
        [ 0.4121, -0.9111,  0.7833,  0.6216,  0.0899,  0.9960]])

# Combining all above cells in a class for reusability

In [25]:
import torch
import torch.nn as nn
class PositionalEncoding(nn.Module):
  def __init__(self, d_model, max_sequence_length):
    super().__init__()
    self.max_sequence_length = max_sequence_length
    self.d_model= d_model
  def forward(self):
    even_i= torch.arange(0, self.d_model, 2).float()
    denominator = torch.pow (10000, even_i/self.d_model)
    position = torch.arange(self.max_sequence_length). reshape (self.max_sequence_length, 1)
    even_PE = torch.sin(position / denominator)
    odd_PE = torch.cos(position / denominator)
    stacked = torch.stack( [even_PE, odd_PE], dim=2)
    PE = torch.flatten (stacked, start_dim=1, end_dim=2)
    return PE

In [26]:
pe=PositionalEncoding(d_model = 6, max_sequence_length = 10)
pe.forward()

tensor([[ 0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.8415,  0.5403,  0.0464,  0.9989,  0.0022,  1.0000],
        [ 0.9093, -0.4161,  0.0927,  0.9957,  0.0043,  1.0000],
        [ 0.1411, -0.9900,  0.1388,  0.9903,  0.0065,  1.0000],
        [-0.7568, -0.6536,  0.1846,  0.9828,  0.0086,  1.0000],
        [-0.9589,  0.2837,  0.2300,  0.9732,  0.0108,  0.9999],
        [-0.2794,  0.9602,  0.2749,  0.9615,  0.0129,  0.9999],
        [ 0.6570,  0.7539,  0.3192,  0.9477,  0.0151,  0.9999],
        [ 0.9894, -0.1455,  0.3629,  0.9318,  0.0172,  0.9999],
        [ 0.4121, -0.9111,  0.4057,  0.9140,  0.0194,  0.9998]])